In [ ]:
!wget https://www.kasprowski.pl/datasets/weather.zip

--2021-11-11 10:25:34--  https://www.kasprowski.pl/datasets/weather.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95380686 (91M) [application/zip]
Saving to: ‘weather.zip’

weather.zip         100%[===================>]  90.96M  11.7MB/s    in 8.3s    

2021-11-11 10:25:43 (10.9 MB/s) - ‘weather.zip’ saved [95380686/95380686]



In [ ]:
!unzip 'weather'

Archive:  weather.zip
   creating: weather/cloudy/
  inflating: weather/cloudy/cloudy1.jpg  
  inflating: weather/cloudy/cloudy10.jpg  
  inflating: weather/cloudy/cloudy100.jpg  
  inflating: weather/cloudy/cloudy101.jpg  
  inflating: weather/cloudy/cloudy102.jpg  
  inflating: weather/cloudy/cloudy103.jpg  
  inflating: weather/cloudy/cloudy104.jpg  
  inflating: weather/cloudy/cloudy105.jpg  
  inflating: weather/cloudy/cloudy106.jpg  
  inflating: weather/cloudy/cloudy107.jpg  
  inflating: weather/cloudy/cloudy108.jpg  
  inflating: weather/cloudy/cloudy109.jpg  
  inflating: weather/cloudy/cloudy11.jpg  
  inflating: weather/cloudy/cloudy110.jpg  
  inflating: weather/cloudy/cloudy111.jpg  
  inflating: weather/cloudy/cloudy112.jpg  
  inflating: weather/cloudy/cloudy113.jpg  
  inflating: weather/cloudy/cloudy114.jpg  
  inflating: weather/cloudy/cloudy115.jpg  
  inflating: weather/cloudy/cloudy116.jpg  
  inflating: weather/cloudy/cloudy117.jpg  
  inflating: weather/cloudy/c

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [4]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if(class_dir == 'shine' or class_dir == 'rain'):
          continue
        if not os.path.isdir(indir+'/'+class_dir): 
          continue
        print("Loading:",class_dir)
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('weather')
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels

Loading: cloudy
Loading: sunrise
loaded 657  samples
classes {'cloudy', 'sunrise'}


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
encoder = sklearn.preprocessing.LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels[:657])

## find text label for label
label = np.array([0,1])
print('Label:',label)
print("Text label:",encoder.inverse_transform(label)) # WRONG! each value treated independently!
print('argmax label:',label.argmax())
#print(encoder.inverse_transform(label.argmax())) # encoder expects colection of values!
print("Text label:",encoder.inverse_transform([label.argmax()])) # works when packed into a collection

One-hot labels [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Label: [0 1]
Text label: ['cloudy' 'sunrise']
argmax label: 1
Text label: ['sunrise']


In [6]:
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

model.summary()
optimizer = tf.keras.optimizers.RMSprop(0.0001, decay = 1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 64, 64, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       1

In [7]:
print(samples.shape)
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples,labels,random_state=22)
model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

(657, 64, 64, 3)


In [8]:
H = model.fit(trainSamples, trainLabels, epochs=10, validation_data=(testSamples,testLabels))

Epoch 1/10
16/16 [==============================] - 4s 204ms/step - loss: 1.7704 - accuracy: 0.8435 - val_loss: 8.4001 - val_accuracy: 0.5818
Epoch 2/10
16/16 [==============================] - 3s 187ms/step - loss: 0.2413 - accuracy: 0.9472 - val_loss: 1.7001 - val_accuracy: 0.7758
Epoch 3/10
16/16 [==============================] - 3s 184ms/step - loss: 0.0306 - accuracy: 0.9837 - val_loss: 0.2737 - val_accuracy: 0.9333
Epoch 4/10
16/16 [==============================] - 3s 186ms/step - loss: 0.0413 - accuracy: 0.9878 - val_loss: 0.2970 - val_accuracy: 0.9636
Epoch 5/10
16/16 [==============================] - 3s 187ms/step - loss: 0.0322 - accuracy: 0.9837 - val_loss: 0.6481 - val_accuracy: 0.9030
Epoch 6/10
16/16 [==============================] - 3s 185ms/step - loss: 0.0092 - accuracy: 0.9939 - val_loss: 0.2774 - val_accuracy: 0.9576
Epoch 7/10
16/16 [==============================] - 3s 185ms/step - loss: 0.0045 - accuracy: 0.9980 - val_loss: 0.2953 - val_accuracy: 0.9515
Epoch 

In [9]:
from sklearn.neighbors import KNeighborsClassifier

res = model.predict(testSamples)    
print(confusion_matrix(testLabels.argmax(axis=1), res.argmax(axis=1)))
print(classification_report(testLabels.argmax(axis=1), res.argmax(axis=1)))
knnAcc = accuracy_score(testLabels.argmax(axis=1), res.argmax(axis=1)) 
print("Cohen's Kappa: {}".format(cohen_kappa_score(testLabels.argmax(axis=1), res.argmax(axis=1))))
print("kNN Accuracy: {:.2f}".format(knnAcc))

[[73  6]
 [ 0 86]]
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        79
           1       0.93      1.00      0.97        86

    accuracy                           0.96       165
   macro avg       0.97      0.96      0.96       165
weighted avg       0.97      0.96      0.96       165

Cohen's Kappa: 0.9269156946700132
kNN Accuracy: 0.96


In [10]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        if not os.path.isdir(indir+'/'+class_dir): 
          continue
        print("Loading:",class_dir)
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('weather')
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels

Loading: cloudy
Loading: sunrise
Loading: shine
Loading: rain
loaded 1123  samples
classes {'cloudy', 'shine', 'sunrise', 'rain'}


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
encoder = sklearn.preprocessing.LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels[:657])

## find text label for label
label = np.array([0,1])
print('Label:',label)
print("Text label:",encoder.inverse_transform(label)) # WRONG! each value treated independently!
print('argmax label:',label.argmax())
#print(encoder.inverse_transform(label.argmax())) # encoder expects colection of values!
print("Text label:",encoder.inverse_transform([label.argmax()])) # works when packed into a collection

One-hot labels [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
Label: [0 1]
Text label: ['cloudy' 'rain']
argmax label: 1
Text label: ['rain']


In [12]:
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(4))
model.add(Activation("softmax"))

model.summary()
optimizer = tf.keras.optimizers.RMSprop(0.0001, decay = 1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 64, 64, 16)        448       
                                                                 
 batch_normalization_2 (Batc  (None, 64, 64, 16)       64        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 64, 64, 16)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_3 (Batc  (None, 32, 32, 32)      

In [13]:
print(samples.shape)
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples,labels,random_state=22)
model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

(1123, 64, 64, 3)


In [14]:
H = model.fit(trainSamples, trainLabels, epochs=10, validation_data=(testSamples,testLabels))

Epoch 1/10
27/27 [==============================] - 6s 189ms/step - loss: 2.9309 - accuracy: 0.6211 - val_loss: 4.9599 - val_accuracy: 0.6299
Epoch 2/10
27/27 [==============================] - 5s 184ms/step - loss: 0.4494 - accuracy: 0.8587 - val_loss: 4.5735 - val_accuracy: 0.5765
Epoch 3/10
27/27 [==============================] - 5s 185ms/step - loss: 0.2940 - accuracy: 0.8872 - val_loss: 2.5350 - val_accuracy: 0.5907
Epoch 4/10
27/27 [==============================] - 5s 185ms/step - loss: 0.1878 - accuracy: 0.9228 - val_loss: 2.2726 - val_accuracy: 0.5872
Epoch 5/10
27/27 [==============================] - 5s 185ms/step - loss: 0.1535 - accuracy: 0.9454 - val_loss: 1.1274 - val_accuracy: 0.7331
Epoch 6/10
27/27 [==============================] - 5s 184ms/step - loss: 0.1640 - accuracy: 0.9418 - val_loss: 0.6326 - val_accuracy: 0.8434
Epoch 7/10
27/27 [==============================] - 5s 184ms/step - loss: 0.1256 - accuracy: 0.9608 - val_loss: 0.8467 - val_accuracy: 0.7829
Epoch 

In [15]:
from sklearn.neighbors import KNeighborsClassifier

res = model.predict(testSamples)    
print(confusion_matrix(testLabels.argmax(axis=1), res.argmax(axis=1)))
print(classification_report(testLabels.argmax(axis=1), res.argmax(axis=1)))
knnAcc = accuracy_score(testLabels.argmax(axis=1), res.argmax(axis=1)) 
print("Cohen's Kappa: {}".format(cohen_kappa_score(testLabels.argmax(axis=1), res.argmax(axis=1))))
print("kNN Accuracy: {:.2f}".format(knnAcc))

[[67  5  4  0]
 [ 1 48  1  0]
 [ 4  2 59  2]
 [ 4  0  3 81]]
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        76
           1       0.87      0.96      0.91        50
           2       0.88      0.88      0.88        67
           3       0.98      0.92      0.95        88

    accuracy                           0.91       281
   macro avg       0.90      0.91      0.91       281
weighted avg       0.91      0.91      0.91       281

Cohen's Kappa: 0.8754135261416732
kNN Accuracy: 0.91
